# Query 2
## With DataFrames

In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1795,application_1765289937462_1779,pyspark,idle,Link,Link,None,
1803,application_1765289937462_1787,pyspark,idle,Link,Link,None,


In [2]:
# We initialized a spark session with specific configurations, now we import
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, to_timestamp, year, count, sum, row_number, round, when
from pyspark.sql.window import Window
import time

#Beginning of timing
start_time = time.time()

# Define schema for crime data DataFrame
crime_data_full_schema = StructType([
    StructField("DR_NO", IntegerType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", IntegerType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", IntegerType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType())
])

# Create DataFrame
crime_data_full_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/", \
    header=True, \
    schema=crime_data_full_schema) 

crime_data_df = crime_data_full_df.select("DR_NO", "Date Rptd", "Vict Descent")

# Find the year
crime_data_with_year_df = crime_data_df.withColumn(
    'Timestamp', to_timestamp(col('Date Rptd'), 'yyyy MMM dd hh:mm:ss a') ) \
    .withColumn('year', year(col('Timestamp')))

analysis_df = crime_data_with_year_df.select("DR_NO", "Vict Descent", "year") \
    .filter(col("year").isNotNull() & col("Vict Descent").isNotNull())

# Group by Year and Descent and then count
grouped_df = analysis_df.groupBy("year", "Vict Descent").agg(
    count(col("DR_NO")).alias("#"))

# Window functions for ranking and total count per year for percentage calculation
window_year_total = Window.partitionBy("year")

window_ranking = Window.partitionBy("year").orderBy(col("#").desc())

ranked_df = grouped_df.withColumn("Total_Year_Count", sum(col("#")).over(window_year_total)) \
    .withColumn("Rank", row_number().over(window_ranking)) \
    .withColumn("%", round((col("#") / col("Total_Year_Count")) * 100, 1))

# Convert Vict Descent to full version
descent_mapping = (
    when(col("Vict Descent") == "A", "Other Asian") 
    .when(col("Vict Descent") == "B", "Black")
    .when(col("Vict Descent") == "C", "Chinese")
    .when(col("Vict Descent") == "D", "Cambodian")
    .when(col("Vict Descent") == "F", "Filipino")
    .when(col("Vict Descent") == "G", "Guamanian")
    .when(col("Vict Descent") == "H", "Hispanic/Latin/Mexican")
    .when(col("Vict Descent") == "I", "American Indian")
    .when(col("Vict Descent") == "J", "Japanese")
    .when(col("Vict Descent") == "K", "Korean")
    .when(col("Vict Descent") == "L", "Laotian")
    .when(col("Vict Descent") == "O", "Other")
    .when(col("Vict Descent") == "P", "Pacific Islander")
    .when(col("Vict Descent") == "S", "Samoan")
    .when(col("Vict Descent") == "U", "Hawaiian")
    .when(col("Vict Descent") == "V", "Vietnamese")
    .when(col("Vict Descent") == "W", "White")
    .when(col("Vict Descent") == "X", "Unknown")
    .when(col("Vict Descent") == "Z", "Asian Indian")
    .otherwise(col("Vict Descent"))
)

ranked_df_mapped = ranked_df.withColumn("Victim Descent", descent_mapping)

# Filtering for final output
final_result_df = ranked_df_mapped.filter(col("Rank") <= 3) \
    .select(
    col("year"),
    col("Victim Descent"), 
    col("#"),
    col("%"),) \
    .orderBy(
    col("year").desc(), 
    col("Rank").asc())

# Show results
final_result_df.show(48, truncate=False)

# End of timing
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} sec")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1805,application_1765289937462_1789,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+----+
|year|Victim Descent        |#    |%   |
+----+----------------------+-----+----+
|2025|Hispanic/Latin/Mexican|70   |38.3|
|2025|White                 |41   |22.4|
|2025|Unknown               |35   |19.1|
|2024|Hispanic/Latin/Mexican|30547|29.1|
|2024|White                 |24758|23.6|
|2024|Unknown               |20784|19.8|
|2023|Hispanic/Latin/Mexican|70920|34.7|
|2023|White                 |45553|22.3|
|2023|Black                 |31247|15.3|
|2022|Hispanic/Latin/Mexican|74061|35.8|
|2022|White                 |47151|22.8|
|2022|Black                 |35064|16.9|
|2021|Hispanic/Latin/Mexican|64333|35.2|
|2021|White                 |44766|24.5|
|2021|Black                 |30423|16.7|
|2020|Hispanic/Latin/Mexican|61840|35.5|
|2020|White                 |42469|24.4|
|2020|Black                 |28395|16.3|
|2019|Hispanic/Latin/Mexican|73260|36.5|
|2019|White                 |49103|24.5|
|2019|Black                 |33395|16.6|
|2018|Hispanic/L

## SQL API

In [3]:
# We initialized a spark session with specific configurations, now we import
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import time

#Beginning of timing
start_time = time.time()

# Define schema for crime data DataFrame
crime_data_full_schema = StructType([
    StructField("DR_NO", IntegerType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", IntegerType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", IntegerType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType())
])

# Create DataFrame
crime_data_full_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/", \
    header=True, \
    schema=crime_data_full_schema) 

crime_data_df = crime_data_full_df.select("DR_NO", "Date Rptd", "Vict Descent")

# Create temporary table for SQL queries
crime_data_df.createOrReplaceTempView("crime_data_table")

sql_query = """
WITH PreprocessedData AS (
    SELECT
        DR_NO,
        `Vict Descent`, 
        
        -- Find the year
        YEAR(TO_TIMESTAMP(`Date Rptd`, 'yyyy MMM dd hh:mm:ss a')) AS year,
        
        -- 2. Descent mapping
        CASE `Vict Descent`
            WHEN 'A' THEN 'Other Asian'
            WHEN 'B' THEN 'Black'
            WHEN 'C' THEN 'Chinese'
            WHEN 'D' THEN 'Cambodian'
            WHEN 'F' THEN 'Filipino'
            WHEN 'G' THEN 'Guamanian'
            WHEN 'H' THEN 'Hispanic/Latin/Mexican'
            WHEN 'I' THEN 'American Indian'
            WHEN 'J' THEN 'Japanese'
            WHEN 'K' THEN 'Korean'
            WHEN 'L' THEN 'Laotian'
            WHEN 'O' THEN 'Other'
            WHEN 'P' THEN 'Pacific Islander'
            WHEN 'S' THEN 'Samoan'
            WHEN 'U' THEN 'Hawaiian'
            WHEN 'V' THEN 'Vietnamese'
            WHEN 'W' THEN 'White'
            WHEN 'X' THEN 'Unknown'
            WHEN 'Z' THEN 'Asian Indian'
            ELSE `Vict Descent`
        END AS `Victim Descent`
    FROM crime_data_table
    
    -- Filter out null values
    WHERE YEAR(TO_TIMESTAMP(`Date Rptd`, 'yyyy MMM dd hh:mm:ss a')) IS NOT NULL
      AND `Vict Descent` IS NOT NULL
),
RankedData AS (
    SELECT
        year,
        `Victim Descent`,
        
        -- Count of incidents for this Descent/Year group
        COUNT(DR_NO) AS `Count_#`,
        
        -- Total Count for the Year (Window Function)
        SUM(COUNT(DR_NO)) OVER (PARTITION BY year) AS Total_Year_Count,
        
        -- Rank within the Year (Window Function)
        ROW_NUMBER() OVER (PARTITION BY year ORDER BY COUNT(DR_NO) DESC) AS Rank
    FROM PreprocessedData
    GROUP BY year, `Victim Descent`
)
SELECT
    year,
    `Victim Descent`,
    `Count_#` AS `#`,
    
    -- Calculate percentage
    ROUND((`Count_#` / Total_Year_Count) * 100, 1) AS `%`
FROM RankedData
WHERE Rank <= 3 -- Final filtering
ORDER BY year DESC, Rank ASC
"""

# SQL Query execution
final_result_sql_df = spark.sql(sql_query)

# Final results
final_result_sql_df.show(48, truncate=False) 

# End of timing
execution_time = time.time() - start_time
print(f"\nExecution time: {execution_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+----+
|year|Victim Descent        |#    |%   |
+----+----------------------+-----+----+
|2025|Hispanic/Latin/Mexican|70   |38.3|
|2025|White                 |41   |22.4|
|2025|Unknown               |35   |19.1|
|2024|Hispanic/Latin/Mexican|30547|29.1|
|2024|White                 |24758|23.6|
|2024|Unknown               |20784|19.8|
|2023|Hispanic/Latin/Mexican|70920|34.7|
|2023|White                 |45553|22.3|
|2023|Black                 |31247|15.3|
|2022|Hispanic/Latin/Mexican|74061|35.8|
|2022|White                 |47151|22.8|
|2022|Black                 |35064|16.9|
|2021|Hispanic/Latin/Mexican|64333|35.2|
|2021|White                 |44766|24.5|
|2021|Black                 |30423|16.7|
|2020|Hispanic/Latin/Mexican|61840|35.5|
|2020|White                 |42469|24.4|
|2020|Black                 |28395|16.3|
|2019|Hispanic/Latin/Mexican|73260|36.5|
|2019|White                 |49103|24.5|
|2019|Black                 |33395|16.6|
|2018|Hispanic/L